**Motor Vehicle Retail Sales: Heavy Weight Trucks**

In [1]:
!pip install pystan==2.19.1.1 --quiet
!pip install fbprophet --quiet
!pip install yfinance --quiet
!pip install Nasdaq-Data-Link

     |████████████████████████████████| 62.6 MB 96 kB/s 
     |████████████████████████████████| 64 kB 1.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prophet 1.1.1 requires cmdstanpy>=1.0.4, but you have cmdstanpy 0.9.5 which is incompatible.
     |████████████████████████████████| 62 kB 1.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import nasdaqdatalink
motor_vehicle_retail_sales_df = nasdaqdatalink.get("FRED/HTRUCKSSAAR", authtoken="ugpNyJHCrKx_PeswJSpx")

In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [4]:
today = datetime.today().strftime('%Y-%m-%d')

motor_vehicle_retail_sales_df.tail()

,Value
Date,
2021-09-01,$0.41
2021-10-01,$0.44
2021-11-01,$0.44
2021-12-01,$0.46
2022-01-01,$0.53


In [5]:
motor_vehicle_retail_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 661 entries, 1967-01-01 to 2022-01-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   661 non-null    float64
dtypes: float64(1)
memory usage: 10.3 KB


In [6]:
motor_vehicle_retail_sales_df.isnull().sum()

Value    0
dtype: int64

In [7]:
motor_vehicle_retail_sales_df.reset_index(inplace=True)
motor_vehicle_retail_sales_df.columns

Index(['Date', 'Value'], dtype='object')

In [8]:
motor_vehicle_retail_sales_df = motor_vehicle_retail_sales_df[["Date", "Value"]]

new_names = {
    "Date": "ds", 
    "Value": "y",
}

motor_vehicle_retail_sales_df.rename(columns=new_names, inplace=True)

In [9]:
motor_vehicle_retail_sales_df.tail()

,ds,y
656,2021-09-01,$0.41
657,2021-10-01,$0.44
658,2021-11-01,$0.44
659,2021-12-01,$0.46
660,2022-01-01,$0.53


In [10]:
# plot the open price

x = motor_vehicle_retail_sales_df["ds"]
y = motor_vehicle_retail_sales_df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Motor Vehicle Retail Sales: Heavy Weight Trucks",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [11]:
model = Prophet(
    seasonality_mode="multiplicative" 
)

model.fit(motor_vehicle_retail_sales_df)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [12]:
future = model.make_future_dataframe(periods = 365)
future.tail()

,ds
1021,2022-12-28
1022,2022-12-29
1023,2022-12-30
1024,2022-12-31
1025,2023-01-01


In [13]:
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1021,2022-12-28,$0.42,$0.32,$0.52
1022,2022-12-29,$0.43,$0.32,$0.53
1023,2022-12-30,$0.43,$0.34,$0.53
1024,2022-12-31,$0.44,$0.34,$0.55
1025,2023-01-01,$0.45,$0.35,$0.55


In [14]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

0.43920464159808364

In [15]:
plot_plotly(model, forecast)

In [16]:
plot_components_plotly(model, forecast)